# ORM Quick Start

- SQLAlchemy는 "파이썬을 위한 데이터베이스 툴킷"으로, 데이터베이스 연결, 쿼리 및 결과 상호 작용, SQL 문 프로그래밍 방식 구성 등을 관리하는 도구를 제공

- Core: 데이터베이스 연결 관리, SQL 쿼리 및 결과 처리, SQL 문장 프로그래밍 방식 구성 등을 위한 기본 아키텍처

- ORM (Object Relational Mapping): Core 위에 구축되어 파이썬 클래스를 데이터베이스 테이블에 매핑하고, 객체 영속성(persistence) 메커니즘인 **세션(Session)**을 제공. 이를 통해 SQL 쿼리를 파이썬 객체 관점에서 작성하고 호출 가능

## 1. 모델 선언 - Declare Models
- ORM 작업의 시작은 데이터베이스 스키마를 파이썬 객체로 정의하는것 -> Declarative Mapping
    1. Base 클래스 정의: 모든 매핑은 DeclarativeBase를 상속받는 Base 클래스로부터 시작
    2. 테이블 이름(`__tablename__`): 각 모델 클래스는 `__tablename__` 이라는 특별한 클래스 속성을 통해 매핑될 데이터베이스 테이블의 이름을 지정 **빠트리면 안됨**
    3. 컬럼 선언 (`Mapped`, `mapped_column`)
        - 테이블의 컬럼들은 `Mapped` __타입 어노테이션__ 을 사용하여 정의된다. EX) `id: Mapped[int]` -> 파이썬 타입이 자동으로 SQL 타입(`INTEGER`, `VARCHAR`) 로 변환
        - `OPTIONAL[]` 해당 컬럼이 NULL값 허용
        - `mapped_column()` 지시문은 더 세부적인 컬럼 설정이 필요할때 사용. `STRING(30)`, `primary_key=True`, `ForeignKey` 와 같은 제약조건
        - 관계 설정: `relationship()`은 두 ORM 클래스 간의 연결을 나타낸다. 
            ```python
            class User(Base):
                __tablename__ = "user_account"

                id: Mapped[int] = mapped_column(primary_key=True)
                name: Mapped[str] = mapped_column(String(30))
                fullname: Mapped[Optional[str]]

                addresses: Mapped[List["Address"]] = relationship(
                    back_populates="user", cascade="all, delete-orphan"
                )

                def __repr__(self) -> str:
                    return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"
            ```
            - 한 명의 `User`는 여러개의 `address`를 가질 수 있기 때문에 `Mapped[List["Address]]` 로 타입힌트
            - `back_populates="user"`: 유저 객체의 addresses와 address 객체의 user와 연결
            -  한쪽에서 객체를 추가하거나 삭제할 때 다른 쪽에서도 자동으로 동기화
            - __repr__() 메서드: 필수는 아니지만, 디버깅할 때 객체를 쉽게 알아볼 수 있도록 해주는 아주 유용한 메서드




---


훈련 과제 1: 나만의 모델 만들기 다음 요구사항에 맞춰 파이썬 코드로 두 개의 ORM 모델을 선언
1. Product 모델:

테이블 이름은 "products"로 지정.

- id: 정수형 기본 키 (자동 증가).

- name: 255자 길이의 문자열, NULL 불가.

- price: 실수형, 기본값은 0.0.

- category_id: 정수형, Category 테이블의 id를 참조하는 외래 키.

- __repr__ 메서드를 포함하여 객체 정보를 쉽게 확인할 수 있도록 할 것.
2. Category 모델:

- 테이블 이름은 "categories"로 지정.

- id: 정수형 기본 키 (자동 증가).

- name: 100자 길이의 문자열, NULL 불가, 고유(unique)해야 함.

- products: Product 모델과의 일대다 관계를 설정할 것. back_populates와 cascade를 적절히 사용해라.

```python
# 필요한 모듈 임포트
# from typing import List, Optional
# from sqlalchemy import ForeignKey, String, Integer, Float
# from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship

# Base 클래스 정의
# class Base(DeclarativeBase):
#     pass

# Product 및 Category 모델 정의
# ... 너의 코드를 여기에 작성해봐!
```

In [ ]:
# 필요한 모듈 임포트
from typing import List, Optional
from sqlalchemy import ForeignKey, String, Integer, Float
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship

# Base 클래스 정의
class Base(DeclarativeBase):
    pass

# Product 및 Category 모델 정의
class Product(Base):
    __tablename__ = "products"
    
    i: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    name: Mapped[str] = mapped_column(String(30))
    price: Mapped[float] = mapped_column(default=0.0)
    category_id: Mapped[int] = mapped_column(ForeignKey("categories.id")) # fk
    category: Mapped["Category"] = relationship(back_populates="products")

    def __repr__(self) -> str:
        return f"Prodict(id={self.id!r}, )"


class Category(Base):
    __tablename__ = "categories"
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    name: Mapped[str] = mapped_column(String(100), unique=True)
    products: Mapped[List["Product"]] = relationship(back_populates="category",
                                                     cascade="all, delete-orphan" )
    def __repr__(self) -> str: 
        return f"Category(id={self.id!r}), name={self.name!r}"


        

Product 테이블이 성공적으로 생성되었습니다!
<Product(id=1, name='노트북', price=1200000)>
